In [1]:
%load_ext autoreload
%autoreload 2
import spike_queries
import pandas as pd
from typing import List, Tuple
import tqdm
import pickle

In [2]:
query =  "arg1:paracetamol $treats arg2:headaches." #"virus lemma=persist on"
dataset_name = "covid19"
num_results = 100
query_type = "syntactic"

#df = spike_queries.perform_query(query, dataset_name, num_results, query_type, lucene_query="(Obama AND AND Hawaii) OR (US)") #previously: word=Hawaii
df = spike_queries.perform_query(query, dataset_name, num_results, query_type)



{"queries": {"syntactic": "arg1:paracetamol $treats arg2:headaches.", "parent":"","expansion":""},"data_set_name":"cord19","context":{"lists":{},"tables":{},"case_strategy":"ignore",
    "attempt_fuzzy": false}}




<Response [204]>


https://spike.staging.apps.allenai.org/api/3/search/query/54c41cb9d5dbded221d0f4b201e3b33a/csv?sentence_text=True&capture_indices=True&sentence_id=True&limit=100


In [55]:
# print(len(df))
# for s in (df["sentence_text"].tolist()):
#     print(("Hawaii" in s and "Obama" in s) or "US" in s)

In [20]:
df.head()

,sentence_id,arg1,arg2,people,title,article_link,arg1_first_index,arg1_last_index,arg2_first_index,arg2_last_index,people_first_index,people_last_index,sentence_text,paragraph_text
0,450,a,display,preprint,Spatial Profiling of Lung SARS-CoV-2 and Influ...,https://api.semanticscholar.org/10.1101/2020.1...,8,8,11,11,13,13,"is the author/funder , who has granted medRxiv...","is the author/funder, who has granted medRxiv ..."
1,481,The,Should,keynote,Innovation and Reform on Technology Empowered ...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,0,0,7,7,2,2,"The second keynote speech , Abilities Teachers...","The second keynote speech, Abilities Teachers ..."
2,481,The,Should,Attain,Innovation and Reform on Technology Empowered ...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,0,0,7,7,8,8,"The second keynote speech , Abilities Teachers...","The second keynote speech, Abilities Teachers ..."
3,623,a,display,preprint,Spatial Profiling of Lung SARS-CoV-2 and Influ...,https://api.semanticscholar.org/10.1101/2020.1...,8,8,11,11,13,13,"is the author/funder , who has granted medRxiv...",The copyright holder for this this version pos...
4,1154,a,display,preprint,Spatial Profiling of Lung SARS-CoV-2 and Influ...,https://api.semanticscholar.org/10.1101/2020.1...,8,8,11,11,13,13,"is the author/funder , who has granted medRxiv...","is the author/funder, who has granted medRxiv ..."


In [3]:
with open("patterns.tsv", "r") as f:
    lines = f.readlines()

In [4]:
patterns_and_relations = [s.strip().split("\t") for s in lines]

In [5]:
patterns_and_relations = patterns_and_relations[:-1]
rel2results = defaultdict(list)


In [13]:
from collections import defaultdict
import time


dataset_name = "wiki"
num_results = 100
query_type = "syntactic"


def call_spike(pattern, dataset_name, num_results, query_type):
    
    return spike_queries.perform_query(pattern, dataset_name, num_results, query_type)

i = 0
for pattern,relation_id in tqdm.tqdm_notebook(patterns_and_relations[901:]):
    
    print(pattern)
    i += 1
    pattern = pattern.replace("arg1", "ARG1").replace("arg2", "ARG2")
    try:
        df = call_spike(pattern, dataset_name, num_results, query_type)
    except Exception as e:
        print("Failed", e)
        failed = True
        q=0
        while failed:
            time.sleep(15)
            q += 1
            if q > 2:
                break
            try:
                df = spike_queries.perform_query(pattern, dataset_name, num_results, query_type)
                failed = False
            except:
                continue
        
            
    try:
        sents = df["sentence_text"].tolist()
        arg1_start_idx = df["ARG1_first_index"].tolist()
        arg1_end_idx = df["ARG1_last_index"].tolist()
    
        arg2_start_idx = df["ARG2_first_index"].tolist()
        arg2_end_idx = df["ARG2_last_index"].tolist()
    
        if len(sents) > 0:
            for s,arg1_start,arg1_end,arg2_start,arg2_end in zip(sents, arg1_start_idx, arg1_end_idx, arg2_start_idx, arg2_end_idx):
        
                rel2results[relation_id].append({"sent": s, "arg1": (arg1_start, arg1_end), 
                                             "arg2": (arg2_start, arg2_end), "pattern": pattern})
    except Exception as e:
        continue
    time.sleep(1)
    
    if i % 1 == 0:
        
        with open("rel2results.pickle", "wb") as f:
            pickle.dump(rel2results, f)

/home/shauli/anaconda3/envs/py3/lib/python3.7/site-packages/ipykernel_launcher.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  from ipykernel import kernelapp as app


  0%|          | 0/1453 [00:00<?, ?it/s]

*ARG1:X $consists $of ARG2:Y.


{"queries": {"syntactic": "*ARG1:X $consists $of ARG2:Y.", "parent":"","expansion":""},"data_set_name":"cord19","context":{"lists":{},"tables":{},"case_strategy":"ignore",
    "attempt_fuzzy": false}}




<Response [204]>


https://spike.staging.apps.allenai.org/api/3/search/query/710c4951fe5fc64327a74313ec06a320/csv?sentence_text=True&capture_indices=True&sentence_id=True&limit=100
*ARG1:X $consisted $of ARG2:Y.


{"queries": {"syntactic": "*ARG1:X $consisted $of ARG2:Y.", "parent":"","expansion":""},"data_set_name":"cord19","context":{"lists":{},"tables":{},"case_strategy":"ignore",
    "attempt_fuzzy": false}}




<Response [204]>


https://spike.staging.apps.allenai.org/api/3/search/query/9dfbf013699f9ec2145808e9c64a2ed8/csv?sentence_text=True&capture_indices=True&sentence_id=True&limit=100
*ARG1:X $comprises ARG2:Y.


{"queries": {"syntactic": "*ARG1:X $comprises ARG2:Y.", "parent":"","expansion":""},"data_set_name":"cord19","context":{"lists":{},"t



<Response [204]>


https://spike.staging.apps.allenai.org/api/3/search/query/34a3877ba58c1dc5073af714b17101cf/csv?sentence_text=True&capture_indices=True&sentence_id=True&limit=100
ARG1:X, $who $played $in ARG2:Y $position.


{"queries": {"syntactic": "ARG1:X, $who $played $in ARG2:Y $position.", "parent":"","expansion":""},"data_set_name":"cord19","context":{"lists":{},"tables":{},"case_strategy":"ignore",
    "attempt_fuzzy": false}}




<Response [204]>


https://spike.staging.apps.allenai.org/api/3/search/query/6aaf4925406ca8c33c9784253a363a82/csv?sentence_text=True&capture_indices=True&sentence_id=True&limit=100
$playing $as $a ARG2:Y, ARG1:X


{"queries": {"syntactic": "$playing $as $a ARG2:Y, ARG1:X", "parent":"","expansion":""},"data_set_name":"cord19","context":{"lists":{},"tables":{},"case_strategy":"ignore",
    "attempt_fuzzy": false}}




<Response [204]>


https://spike.staging.apps.allenai.org/api/3/search/query/40ed9db83129dee77d78785aba8082bd/csv?sentence_text=True&c

$the $occupation $of ARG1:X $was ARG2:Y.


{"queries": {"syntactic": "$the $occupation $of ARG1:X $was ARG2:Y.", "parent":"","expansion":""},"data_set_name":"cord19","context":{"lists":{},"tables":{},"case_strategy":"ignore",
    "attempt_fuzzy": false}}




<Response [204]>


https://spike.staging.apps.allenai.org/api/3/search/query/abc04f330772769f0db953f565e6c1de/csv?sentence_text=True&capture_indices=True&sentence_id=True&limit=100
$the $occupation $of ARG1:X $is ARG2:Y.


{"queries": {"syntactic": "$the $occupation $of ARG1:X $is ARG2:Y.", "parent":"","expansion":""},"data_set_name":"cord19","context":{"lists":{},"tables":{},"case_strategy":"ignore",
    "attempt_fuzzy": false}}




<Response [204]>


https://spike.staging.apps.allenai.org/api/3/search/query/487f29e6dab88699389d09921221cb78/csv?sentence_text=True&capture_indices=True&sentence_id=True&limit=100
ARG1:X's $profession $was $a ARG2:Y.


{"queries": {"syntactic": "ARG1:X's $profession $was $a ARG2:Y.", "parent":"","expa

...



<Response [204]>


https://spike.staging.apps.allenai.org/api/3/search/query/66a9f0649b774f47b2d46ec219d828d3/csv?sentence_text=True&capture_indices=True&sentence_id=True&limit=100
ARG1:X, $which $was $held $at $the ARG2:[e=LOC]Y.


{"queries": {"syntactic": "ARG1:X, $which $was $held $at $the ARG2:[e=LOC]Y.", "parent":"","expansion":""},"data_set_name":"cord19","context":{"lists":{},"tables":{},"case_strategy":"ignore",
    "attempt_fuzzy": false}}




<Response [204]>


https://spike.staging.apps.allenai.org/api/3/search/query/7720541f1e20a9d2887ff77d2cbb7c6f/csv?sentence_text=True&capture_indices=True&sentence_id=True&limit=100
ARG1:X, $which $took $place $in ARG2:[e=LOC]Y.


{"queries": {"syntactic": "ARG1:X, $which $took $place $in ARG2:[e=LOC]Y.", "parent":"","expansion":""},"data_set_name":"cord19","context":{"lists":{},"tables":{},"case_strategy":"ignore",
    "attempt_fuzzy": false}}




<Response [204]>


https://spike.staging.apps.allenai.org/api/3/search/query/db43cfc2059



<Response [204]>


https://spike.staging.apps.allenai.org/api/3/search/query/41fb2a3dc64ca643411c52202d805be8/csv?sentence_text=True&capture_indices=True&sentence_id=True&limit=100
ARG1:X, $who $is $employed $by ARG2:Y.


{"queries": {"syntactic": "ARG1:X, $who $is $employed $by ARG2:Y.", "parent":"","expansion":""},"data_set_name":"cord19","context":{"lists":{},"tables":{},"case_strategy":"ignore",
    "attempt_fuzzy": false}}




<Response [204]>


https://spike.staging.apps.allenai.org/api/3/search/query/8d4daf3a95f7ecf03fb21c5bb7225cf4/csv?sentence_text=True&capture_indices=True&sentence_id=True&limit=100
ARG2:Y $CEO ARG1:X.


{"queries": {"syntactic": "ARG2:Y $CEO ARG1:X.", "parent":"","expansion":""},"data_set_name":"cord19","context":{"lists":{},"tables":{},"case_strategy":"ignore",
    "attempt_fuzzy": false}}




<Response [204]>


https://spike.staging.apps.allenai.org/api/3/search/query/1d9c7c5f486a910786a1275f41893857/csv?sentence_text=True&capture_indices=True&sentence_i



<Response [204]>


https://spike.staging.apps.allenai.org/api/3/search/query/4f7f7f1b2559300abe7b4aba2b34b545/csv?sentence_text=True&capture_indices=True&sentence_id=True&limit=100
ARG1:X, $who $is $a $presenter $on ARG2:Y.


{"queries": {"syntactic": "ARG1:X, $who $is $a $presenter $on ARG2:Y.", "parent":"","expansion":""},"data_set_name":"cord19","context":{"lists":{},"tables":{},"case_strategy":"ignore",
    "attempt_fuzzy": false}}




<Response [204]>


https://spike.staging.apps.allenai.org/api/3/search/query/0f64458f7873335db6879caa45562805/csv?sentence_text=True&capture_indices=True&sentence_id=True&limit=100
ARG1:[e=PERSON]X, $who $is $a $presenter $on ARG2:Y.


{"queries": {"syntactic": "ARG1:[e=PERSON]X, $who $is $a $presenter $on ARG2:Y.", "parent":"","expansion":""},"data_set_name":"cord19","context":{"lists":{},"tables":{},"case_strategy":"ignore",
    "attempt_fuzzy": false}}




<Response [204]>


https://spike.staging.apps.allenai.org/api/3/search/query/7a4c6772dd8e2

ARG1:X, $which $is $named $after ARG2:Y.


{"queries": {"syntactic": "ARG1:X, $which $is $named $after ARG2:Y.", "parent":"","expansion":""},"data_set_name":"cord19","context":{"lists":{},"tables":{},"case_strategy":"ignore",
    "attempt_fuzzy": false}}




<Response [204]>


https://spike.staging.apps.allenai.org/api/3/search/query/2fa8aea98d4a8c4ba5d847759cfe986b/csv?sentence_text=True&capture_indices=True&sentence_id=True&limit=100
ARG1:X, $which $was $named $after ARG2:Y.


{"queries": {"syntactic": "ARG1:X, $which $was $named $after ARG2:Y.", "parent":"","expansion":""},"data_set_name":"cord19","context":{"lists":{},"tables":{},"case_strategy":"ignore",
    "attempt_fuzzy": false}}




<Response [204]>


https://spike.staging.apps.allenai.org/api/3/search/query/6f7fb8b2b0ec6ce7d316c255941e8686/csv?sentence_text=True&capture_indices=True&sentence_id=True&limit=100
ARG1:X, $which $is $named $for ARG2:Y.


{"queries": {"syntactic": "ARG1:X, $which $is $named $for ARG2:Y.", "parent":

ARG2:Y $is $the $capital $city $of ARG1:X.


{"queries": {"syntactic": "ARG2:Y $is $the $capital $city $of ARG1:X.", "parent":"","expansion":""},"data_set_name":"cord19","context":{"lists":{},"tables":{},"case_strategy":"ignore",
    "attempt_fuzzy": false}}




<Response [204]>


https://spike.staging.apps.allenai.org/api/3/search/query/bfa8f8d6efe525fc6ba2729325348378/csv?sentence_text=True&capture_indices=True&sentence_id=True&limit=100
ARG1:X's $capital, ARG2:Y.


{"queries": {"syntactic": "ARG1:X's $capital, ARG2:Y.", "parent":"","expansion":""},"data_set_name":"cord19","context":{"lists":{},"tables":{},"case_strategy":"ignore",
    "attempt_fuzzy": false}}




<Response [204]>


https://spike.staging.apps.allenai.org/api/3/search/query/2008c1048570f7e9bbac2eb4e469cdbe/csv?sentence_text=True&capture_indices=True&sentence_id=True&limit=100
ARG1:X's $capital $city, ARG2:Y.


{"queries": {"syntactic": "ARG1:X's $capital $city, ARG2:Y.", "parent":"","expansion":""},"data_set_name":"co

In [14]:
with open("rel2results.pickle", "wb") as f:
            pickle.dump(rel2results, f)

In [31]:
def create_match_object(
    words: List[str], captures_arg1: List[Tuple] = None, captures_arg2: List[Tuple] = None
):

    sent_obj = None
    captures = (
        None
    )
    # print(captures_arg1)
    # print("=========================")
    # print(captures_arg2)
    highlights = [None, None]
    continuation_token = None
    match = None

    return None

def yield_matches(augmented_result_sents, spike_df, alignment_model):
    start = time.time()
    print("time: {}".format(start))
    
    for sentence in augmented_result_sents:

        capture_idx = alignment_supervised.main(alignment_model, [sentence], spike_df[:1], num_results=1, max_ngrams=8)
        #print(capture_idx)
        #print("===================================================")
        match = create_match_object(sentence.split(" "), capture_idx[0][0], capture_idx[0][1])
        print("time: {}; diff: {}".format(time.time(), time.time() - start))
        start = time.time()
        yield match

In [32]:
import alignment_supervised

2021-05-31 10:28:05.398 INFO    faiss.loader: Loading faiss with AVX2 support.
2021-05-31 10:28:05.412 INFO    faiss.loader: Successfully loaded faiss with AVX2 support.


In [34]:
model = alignment_supervised.BertModel("cpu", load_existing= False)

Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [37]:
import time
start = time.time()
iterator = yield_matches(sents, df, model)
print(time.time() - start)

8.106231689453125e-05


In [52]:
start = time.time()
next(iterator)
print(time.time() - start)

time: 1622447116.8352666; diff: 1.0011122226715088
0.15197515487670898


NameError: name 'np' is not defined

In [ ]:
cov_proj